In [1]:
import pandas as pd
import pyodbc
# File names and directories
survey_dir = 'J:/Projects/Surveys/HHTravel/Survey2019/Data/PSRC_2019_HTS_Deliverable_022020/Weighted_Data_022020/'
hh_file_name = '1_household.csv'
person_file_name = '2_person.csv'
vehicle_file_name = '3_vehicle.csv'
day_file_name = '4_day.csv'
trip_file_name = '5_trip.csv'

#sql_conn = pyodbc.connect('DRIVER={SQL Server}; SERVER=AWS-PROD-SQL\COHO;DATABASE=HouseholdTravelSurvey2019;trusted_connection=true')
#trip_table_name = "HHSurvey.Trip"
#trip  = pd.read_sql('SELECT * FROM '+trip_table_name, con = sql_conn)

hh_weight_2017 = 'hh_wt_revised'
hh_weight_2019 = 'hh_wt_2019'
hh_weight_combined = 'hh_wt_combined'
hh_weight_names =[hh_weight_2017, hh_weight_2019, hh_weight_combined]

day_weight_2017 = 'hh_day_wt_revised'
day_weight_2019 = 'hh_day_wt_2019'
day_weight_combined = 'hh_day_wt_combined'
day_weight_names =[day_weight_2017, day_weight_2019, day_weight_combined]

trip_weight_2017 = 'trip_weight_revised'
trip_weight_2019 ='trip_wt_2019'
trip_weight_combined = 'trip_wt_combined'
trip_weight_names =[trip_weight_2017, trip_weight_2019, trip_weight_combined]

codebook_file = 'Combined_Codebook_022020.xlsx'
output_file_hh = 'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/hh_summary.xlsx'
output_file_person= 'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/person_summary.xlsx'
output_file_day= 'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/day_summary.xlsx'
output_file_trip= 'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/trip_summary.xlsx'

In [2]:
def merge_hh_person_trip(hh,person,trip):
    hh_person =pd.merge(hh, person, on= 'hhid', suffixes=['', 'person'], how ='right')
    hh_person_trip = pd.merge(hh_person, trip,on='personid', suffixes=['','trip'], how ='right')
    return hh_person_trip

def merge_hh_person(hh, person):
    hh_person =pd.merge(hh, person, on= 'hhid', suffixes=['', 'person'], how ='right')
    return hh_person


In [3]:
hh = pd.read_csv(survey_dir+hh_file_name, encoding = 'latin-1')
person= pd.read_csv(survey_dir+person_file_name, encoding = 'latin-1')
trip = pd.read_csv(survey_dir+trip_file_name, encoding = 'latin-1')
day= pd.read_csv(survey_dir+day_file_name, encoding='latin-1')

C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (8,44,45,53,56,57,65,72,73,81) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,10,11,20,24,37,39,106,112,118,120,121,128) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (14,15,21,23,34,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
codebook_values = pd.read_excel(survey_dir+codebook_file, sheetname='Values')
codebook_variables =pd.read_excel(survey_dir+codebook_file, sheet_name='Data Overview')

C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [5]:
person_detail = merge_hh_person(hh, person)
trip_detail = merge_hh_person_trip(hh, person, trip)

In [6]:
person_detail['hh_wt_revised'].sum()

3846534.75774668

In [7]:
person_detail['hh_wt_2019'].sum()

4051429.1495613307

In [8]:
person_detail['hh_wt_combined'].sum()

4050211.7293452667

In [9]:
trip_detail['trip_wt_combined'].sum()

17700251.336261034

In [10]:
trip_detail['trip_wt_2019'].sum()

17774204.311235707

In [11]:
def compare_weighted_table(codebook_values, dataset, item, weight_names):
    dataset=dataset.dropna(subset=[item]).reset_index()
    grouped_total=dataset.groupby(item).sum()[weight_names].reset_index()
    for weight_name in weight_names:
        grouped_sum = grouped_total.sum()[weight_name]
        grouped_total[weight_name+'_share']=grouped_total[weight_name]/grouped_sum
   

   
    values=codebook_values.loc[codebook_values['variable']==item]
    variable =codebook_variables.loc[codebook_variables['variable']==item, 'description'].values[0]
    missing_values_df = pd.DataFrame({'variable': [item, item, item],'value':[-9999,-9998, 995], 'label':
                                     ['Missing:Technical Error', 'Missing: Non-response', 'Missing:Skip logic']})
    values = values.append(missing_values_df)
    summary=pd.merge(values, grouped_total,left_on='value', right_on=item)
    summary['variable']=variable

    return summary

In [13]:
hh_vars =['final_home_rgcnum', 'hhsize', 'hhincome_broad', 'prev_res_factors_housing_cost', 'prev_res_factors_income_change']
person_vars=['age', 'gender', 'worker', 'race_afam', 'race_asian', 'mode_freq_1', 'mode_freq_2', 'av_concern_1']
day_vars =['delivery_pkgs_freq', 'delivery_grocery_freq', 'delivery_food_freq','delivery_work_freq','online_shop_time']
trip_vars =['d_purpose', 'd_purp_cat', 'mode_1', 'mode_acc' ]

with pd.ExcelWriter(output_file_hh) as writer:  
    for hh_var in hh_vars:
        compare_weighted_table(codebook_values, hh, hh_var, hh_weight_names).to_excel(writer,sheet_name=hh_var)
        
with pd.ExcelWriter(output_file_person) as writer:  
    for person_var in person_vars:
        compare_weighted_table(codebook_values, person, person_var,hh_weight_names).to_excel(writer,sheet_name=person_var)
        
with pd.ExcelWriter(output_file_day) as writer:  
    for day_var in day_vars:
        compare_weighted_table(codebook_values, day, day_var, day_weight_names).to_excel(writer,sheet_name=day_var)
        
with pd.ExcelWriter(output_file_trip) as writer:  
    for trip_var in trip_vars:
        compare_weighted_table(codebook_values, trip_detail, trip_var, trip_weight_names).to_excel(writer,sheet_name=trip_var)

In [ ]:
compare_weighted_table(codebook_values, person, 'commute_mode', hh_weight_names).to_clipboard()

In [ ]:
pd.DataFrame(trip.columns).to_clipboard()

In [ ]:
person_trip = merge_hh_person_trip(hh,person,trip)

In [ ]:
work_trips=person_trip[(person_trip['o_purpose']==1) & (person_trip['d_purpose']==10)]

In [ ]:
grouped_total=work_trips.groupby(['commute_mode','mode_1']).sum()['trip_weight_revised'].reset_index()

In [ ]:
def lookup_codes(grouped_total, item):
    values=codebook_values.loc[codebook_values['variable']==item]
    variable =codebook_variables.loc[codebook_variables['variable']==item, 'description'].values[0]
    missing_values_df = pd.DataFrame({'variable': [item, item, item],'value':[-9999,-9998, 995], 'label':
                                         ['Missing:Technical Error', 'Missing: Non-response', 'Missing:Skip logic']})
    values = values.append(missing_values_df)
    summary=pd.merge(values, grouped_total,left_on='value', right_on=item)
    summary[item+'_variable']=variable
    return summary

In [ ]:
commute_mode_df = lookup_codes(grouped_total,'commute_mode')
commute_trip_df = lookup_codes(commute_mode_df, 'mode_1')

In [ ]:
commute_trip_df.to_clipboard()

In [15]:
day.groupby(['delivery_pkgs_freq']).sum()[day_weight_2019]

delivery_pkgs_freq
0.0      8.115029e+05
1.0      2.875785e+05
2.0      4.820425e+04
3.0      1.533521e+04
4.0      2.598697e+02
5.0      9.389906e+01
995.0    2.888454e+06
Name: hh_day_wt_2019, dtype: float64